# RoomserviceRclone Service

The RoomserviceRcloneFS manages an Rclone FUSE file system mounted from another context controlled by Roomservice. 

Basic file system access is via pyfilesystem's OSFS. Rclone utilities are accessed via Roomservice API calls.

For example, `getinfo(path, namespaces=['statvfs'])` needs to query Rclone using Rclone utilities in order to receive more accurate information than FUSE provides.


## Rclone FUSE Mounts




In [1]:
from fs.osfs import OSFS
from fs.errors import CreateFailed
from fs.info import Info
from socket import gethostname
from urllib.parse import urlparse
import six
from os import statvfs

import logging
import subprocess
import json
import socket

HT_RCVSIZE = 1024
HT_SOCKADDR = './socket/rs/roomservice.sock'

class Roomservice():
    def call_roomservice(self, message_jsonable):
        # caution: this can handle secrets in the messages.
        #    |     do not log messages in the clear.
        
        logging.debug(message_jsonable)
        
        msg = json.dumps(message_jsonable)
        logging.debug(msg)
    
        message = json.dumps(message_jsonable)
        sock = socket.socket(socket.AF_UNIX, socket.SOCK_STREAM)
        try:
            sock.connect(HT_SOCKADDR)
        except Exception as e:
            logging.debug(e)
            raise
        try:
            logging.debug(f'message to roomservice sent.')
            sock.sendall(message.encode())
            data = sock.recv(HT_RCVSIZE)
            response = data.decode()
            logging.debug(f"response from roomservice received: {response}")
        except Exception as e:
            logging.debug(e)
            sock.close()
            raise
        finally:
            sock.close()
        try:
            return json.loads(response)
        except:
            return "roomserviced returned a response that could not be loaded by the json.loads() method."

    def run(self,
            args, 
            stdin=None, 
            input=None,
            stdout=None,
            stderr=None,
            capture_output=False,
            shell=False,
            cwd=None,
            timeout=None,
            check=False,
            encoding=None,
            errors=None,
            text=None,
            env=None,
            universal_newlines=None,
            **other_popen_kwargs):
        
        return self.call_roomservice({'subprocess.run':
                                      {
                                          'args':args,
                                          'capture_output':capture_output,
                                          'check':check
                                      }
                                     })


def generateShareId(hostname, sharename):
    return sharename +'.'+ hostname

class RoomserviceRcloneFS(OSFS,Roomservice):
    
    
    ###### class methods ######

    # class method
    def initHost(hostname=None):
        """
        0.1
        The BlackstrapFS serves files from
        a virtual context. The context is
        kept inside the class.
        """
        if hostname is None:
            BlackstrapFS.__hostname = gethostname()
        else:
            BlackstrapFS.__hostname = hostname
        BlackstrapFS.__shares = dict()
        
        return BlackstrapFS
    
    
    
    # class method
    def addShare(srcaddr,
                 sharename
                ):
        """
        0.1
        Shares are added to the class itself.
        An instance of the service can
        select which share to serve.
        """
        
        #todo: check for blank hostname
        #todo: check for existing before...
        
        shares = BlackstrapFS.__shares
        
        shareid = generateShareId( 
            BlackstrapFS.__hostname, 
            sharename)
        
        shares[shareid] = srcaddr
        
        return BlackstrapFS
    
    # class method
    def closeHost():
        pass
    
    ###### Instance methods ######
    
    def __init__(self,urlstr,*args):
        # todo: require file scheme
        self._urlstr = urlstr
        
        url = urlparse(urlstr)
        self.url = url
        self.path = url.path
                
        super().__init__(url.path,*args)
        
    def getinfo(self, path='/',namespaces=None):
        info_raw = super().getinfo(path, namespaces).raw
        if namespaces != None and 'limits' in namespaces:
            res = self.run(['rclone','about','--json',path],capture_output=True)
            if res != None and "Good dog" in res:
                
                info_raw['limits'] = {
                    self._urlstr: {
                        'total': res["Good dog"]["total"],
                        'used': res["Good dog"]["used"],
                        'free': res["Good dog"]["free"]                        
                    }
                }
        return Info(info_raw)
        
        
    def geturl(self,path):
        # if path starts with '/'
        _urlstr = self._urlstr.rstrip('/')
        _path = path.lstrip('/')
        if(_path == ""):
            # if there is no path,
            # we must return _urlstr as inited.
            # It may match an entry in the fstab.
            return self._urlstr
        return f"{_urlstr}/{_path}"
        
        
    def _close(self):
        pass
    
    if six.PY2:
        def close(self):  # noqa: D102
            self._close()
            super(BlackstrapFS, self).close()
    else:
        def close(self): # noqa: D102
            self._close()
            super().close()
                    
            
    ####################
    #  cosmos API      #
    ####################
    
    # DEPRICATE THIS
    # DO NOT ADD EXTERNAL API

    def lookup(self, path, base=None):
        # FIXME: Placeholder.
        return True
    

In [2]:
from sys import stdout
logging.basicConfig(stream=stdout,encoding='utf-8',level=logging.DEBUG)

In [3]:
rsfs = RoomserviceRcloneFS('fs')

In [4]:
rsfs.run(['rclone','about','--json','/'],capture_output=True,check=True)

DEBUG:root:{'subprocess.run': {'args': ['rclone', 'about', '--json', '/'], 'capture_output': True, 'check': True}}
DEBUG:root:{"subprocess.run": {"args": ["rclone", "about", "--json", "/"], "capture_output": true, "check": true}}
DEBUG:root:message to roomservice sent.
DEBUG:root:response from roomservice received: {"Good dog": {"total": 121938661376, "used": 99914076160, "free": 15759032320}}



{'Good dog': {'total': 121938661376, 'used': 99914076160, 'free': 15759032320}}

In [5]:
rsfs.getinfo('/',namespaces=['limits']).raw

DEBUG:root:{'subprocess.run': {'args': ['rclone', 'about', '--json', '/'], 'capture_output': True, 'check': False}}
DEBUG:root:{"subprocess.run": {"args": ["rclone", "about", "--json", "/"], "capture_output": true, "check": false}}
DEBUG:root:message to roomservice sent.
DEBUG:root:response from roomservice received: {"Good dog": {"total": 121938661376, "used": 99914076160, "free": 15759032320}}



{'basic': {'name': '', 'is_dir': True},
 'limits': {'fs': {'total': 121938661376,
   'used': 99914076160,
   'free': 15759032320}}}

In [7]:
hostname = gethostname()

BlackstrapFS.initHost('localhost')
BlackstrapFS.addShare(
    srcaddr = 'fs',
    sharename = 'fs'
).addShare(
    srcaddr = 'fs2',
    sharename = 'fs2'
)

fs1 = BlackstrapFS('file://fs.localhost/')
fs1.getinfo('/').raw


NameError: name 'BlackstrapFS' is not defined

In [12]:
fs1.getinfo('/', namespaces=['limits']).raw

{'basic': {'name': '', 'is_dir': True},
 'limits': {'file://fs.localhost/': {'total': 121938661376,
   'used': 99794636800,
   'free': 22144024576}}}

In [9]:
from fs.opener import open
from socket import gethostname

print(gethostname())
open('file://fs')

mbp-linux


(OSFS('/home/raygan/Cosms/Dboy/Laydbug/dev/cburnfs_py/cburnfs/fs'), None)